<a href="https://colab.research.google.com/github/george-zakharov/ml-random/blob/master/Embeddings_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load libs

In [0]:
# Load TF 2.x
%tensorflow_version 2.x

In [4]:
import tensorflow
print(tensorflow.__version__)

2.2.0-rc4


In [14]:
from numpy import array

from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Data

In [0]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic',
]

In [0]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

# Data Processing

In [0]:
all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

In [17]:
unique_words = set(all_words)
print(len(unique_words))

45


In [0]:
vocab_length = 50

In [19]:
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
print(embedded_sentences)

[[48, 9, 34, 34, 3], [9, 1, 39, 38, 34, 43, 14], [28, 4, 1, 14, 9, 34], [37, 3], [20, 24, 3, 12, 34, 43, 14], [42, 7, 38, 6], [49, 30, 3, 7, 26, 3], [14, 9, 7, 4, 3], [35, 49], [34, 14, 6], [21, 44], [14, 39, 29, 6], [34, 21, 18, 43, 9, 3], [9, 3, 39, 35], [34, 41, 18, 14], [9, 49, 9, 21]]


In [0]:
# Let's count max sent vector size
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [21]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[48  9 34 34  3  0  0]
 [ 9  1 39 38 34 43 14]
 [28  4  1 14  9 34  0]
 [37  3  0  0  0  0  0]
 [20 24  3 12 34 43 14]
 [42  7 38  6  0  0  0]
 [49 30  3  7 26  3  0]
 [14  9  7  4  3  0  0]
 [35 49  0  0  0  0  0]
 [34 14  6  0  0  0  0]
 [21 44  0  0  0  0  0]
 [14 39 29  6  0  0  0]
 [34 21 18 43  9  3  0]
 [ 9  3 39 35  0  0  0]
 [34 41 18 14  0  0  0]
 [ 9 49  9 21  0  0  0]]


# Model

In [0]:
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 20)             1000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 140)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 141       
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________
None


# Train

In [25]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6947 - acc: 0.3750
Epoch 2/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6916 - acc: 0.4375
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6886 - acc: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 953us/step - loss: 0.6855 - acc: 0.5625
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6824 - acc: 0.6250
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6794 - acc: 0.6875
Epoch 7/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6763 - acc: 0.7500
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6733 - acc: 0.8125
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6702 - acc: 0.8750
Epoch 10/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6671 - acc: 0.8750
Epoch 11/100
1/1 [==============================] - 0s 2ms/step - loss: 0.664

In [26]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 93.750000


# Notes

https://stackabuse.com/python-for-nlp-word-embeddings-for-deep-learning-in-keras/